In [1]:
import os
path = os.path.abspath(os.getcwd())
path = path[:path.rfind('\\') + 1]
DATA_PATH = path + "data\\"

In [2]:
# DATA_PATH = "D:/github/AML_DS_1/data/collaborative-filtering/"

In [3]:
import pandas as pd
import numpy as np

In [4]:
test = pd.read_csv(DATA_PATH + 'test.csv')
train = pd.read_csv(DATA_PATH + 'train.csv')

In [5]:
ratings = train['rating'].values
userIds = train['userId'].values
itemIds = train['movieId'].values


Implement a basic Collaborative Filtering model.

In [6]:
from scipy.sparse import coo_matrix

n_users = np.max(userIds) + 1
n_items = np.max(itemIds) + 1

R = coo_matrix((ratings, (userIds, itemIds)), shape=(n_users, n_items))

In [7]:
n_features = 15

P = np.random.random((n_users,n_features))
Q = np.random.random((n_items,n_features))

lr = 160
l2 = 0.000001

#train loop
for e in range(250):
    P_tau = P[userIds,:]
    Q_tau = Q[itemIds,:]
    pred = np.sum(P_tau * Q_tau, axis = 1)#np.inner(P_tau, Q_tau)
    R_hat = coo_matrix((pred, (userIds, itemIds)), shape=(n_users, n_items))
    MSE = np.sum(np.square(ratings - pred))/ratings.shape[0] + l2 * (np.sum(np.square(Q)) + np.sum(np.square(P)))
    P -= lr * ((R_hat - R) @ Q)/ratings.shape[0] + l2 * np.square(P)
    Q -= lr * ((R_hat - R).T @ P)/ratings.shape[0] + l2 * np.square(Q)
    lr *= 0.998
    print(f"epoch {e}: MSE = {MSE}, lr = {lr}")

epoch 0: MSE = 2.521747633619708, lr = 159.68
epoch 1: MSE = 1.9988046006617564, lr = 159.36064000000002
epoch 2: MSE = 1.841907768948729, lr = 159.04191872
epoch 3: MSE = 1.755406206445886, lr = 158.72383488256
epoch 4: MSE = 1.6979857562988547, lr = 158.40638721279487
epoch 5: MSE = 1.6560620875959688, lr = 158.08957443836928
epoch 6: MSE = 1.6236869307936002, lr = 157.77339528949256
epoch 7: MSE = 1.5976582223206295, lr = 157.45784849891356
epoch 8: MSE = 1.5761514121635107, lr = 157.14293280191575
epoch 9: MSE = 1.557954388901553, lr = 156.8286469363119
epoch 10: MSE = 1.5423093124961418, lr = 156.51498964243928
epoch 11: MSE = 1.5286352671788317, lr = 156.20195966315438
epoch 12: MSE = 1.5165664184427587, lr = 155.88955574382808
epoch 13: MSE = 1.5057808081957271, lr = 155.57777663234043
epoch 14: MSE = 1.4960865175179654, lr = 155.26662107907575
epoch 15: MSE = 1.4872859959573403, lr = 154.9560878369176
epoch 16: MSE = 1.479273183464635, lr = 154.64617566124375
epoch 17: MSE = 1.

epoch 138: MSE = 1.3227091518634704, lr = 121.13376659367694
epoch 139: MSE = 1.3224235469751466, lr = 120.89149906048958
epoch 140: MSE = 1.3221400091447908, lr = 120.6497160623686
epoch 141: MSE = 1.321858483361789, lr = 120.40841663024386
epoch 142: MSE = 1.321578916105289, lr = 120.16759979698338
epoch 143: MSE = 1.3213012552952934, lr = 119.92726459738941
epoch 144: MSE = 1.3210254502457341, lr = 119.68741006819464
epoch 145: MSE = 1.3207514516194174, lr = 119.44803524805825
epoch 146: MSE = 1.3204792113847592, lr = 119.20913917756214
epoch 147: MSE = 1.320208682774251, lr = 118.97072089920701
epoch 148: MSE = 1.3199398202445494, lr = 118.7327794574086
epoch 149: MSE = 1.3196725794381399, lr = 118.49531389849378
epoch 150: MSE = 1.3194069171465046, lr = 118.25832327069679
epoch 151: MSE = 1.3191427912747051, lr = 118.0218066241554
epoch 152: MSE = 1.3188801608073635, lr = 117.78576301090709
epoch 153: MSE = 1.3186189857759336, lr = 117.55019148488527
epoch 154: MSE = 1.31835922722

In [8]:
ratings_test = test['rating'].values
userIds_test = test['userId'].values
itemIds_test = test['movieId'].values
P_tau = P[userIds_test,:]
Q_tau = Q[itemIds_test,:]
pred = np.sum(P_tau * Q_tau, axis = 1)
MSE_test_MF = np.sum(np.square(ratings_test - pred))/ratings_test.shape[0]


Deep Learning based Collaborative Filtering model.

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import pandas as pd

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

batch_sz = 128

In [10]:
n_samples = len(ratings)

In [11]:
batches = []

#Create batches
for i in range(0, n_samples, batch_sz):
    limit =  min(i + batch_sz, n_samples)
    users_batch, movies_batch, rates_batch = userIds[i: limit], itemIds[i: limit], ratings[i: limit]
    batches.append((torch.tensor(users_batch, dtype=torch.long), torch.tensor(movies_batch, dtype=torch.long),
                  torch.tensor(rates_batch, dtype=torch.float)))

In [12]:
class RecommenderNet(nn.Module):
    def __init__(self, n_users, n_movies, n_factors=50, embedding_dropout=0.02, dropout_rate=0.2):
        super().__init__()

        self.u = nn.Embedding(n_users, n_factors)
        self.m = nn.Embedding(n_movies, n_factors)
        self.drop = nn.Dropout(embedding_dropout)
        self.hidden = nn.Sequential(nn.Linear(2*n_factors, n_factors*4),
                                nn.ReLU(),
                                nn.Dropout(0.15),
                                nn.Linear(n_factors*4, 2*n_factors),
                                nn.ReLU())
        self.fc = nn.Linear(n_factors*2, 1)
        self._init()

    def forward(self, users, movies, minmax=[1,5]):
        features = torch.cat([self.u(users), self.m(movies)], dim=1)
        x = self.drop(features)
        x = self.hidden(x)
        out = torch.sigmoid(self.fc(x))

        if minmax is not None: #Scale the output to [1,5]
            min_rating, max_rating = minmax
            out = (max_rating - min_rating)*out + min_rating
        return out

    def _init(self):
        """
        Initialize embeddings and hidden layers weights with xavier.
        """
        def init(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)

        self.u.weight.data.uniform_(-0.05, 0.05)
        self.m.weight.data.uniform_(-0.05, 0.05)
        self.hidden.apply(init)
        init(self.fc)

In [13]:
net = RecommenderNet(n_factors = 20, n_users=n_users, n_movies=n_items).to(device)
net

RecommenderNet(
  (u): Embedding(6744, 20)
  (m): Embedding(118697, 20)
  (drop): Dropout(p=0.02, inplace=False)
  (hidden): Sequential(
    (0): Linear(in_features=40, out_features=80, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.15, inplace=False)
    (3): Linear(in_features=80, out_features=40, bias=True)
    (4): ReLU()
  )
  (fc): Linear(in_features=40, out_features=1, bias=True)
)

In [14]:
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.3, patience=2)

In [15]:
epochs = 20

for epoch in range(epochs):
    train_loss = 0
    c = 0
    for users_batch, movies_batch, rates_batch in batches:
        net.zero_grad()
        out = net(users_batch.to(device), movies_batch.to(device), [1, 5]).squeeze()
        loss = criterion(rates_batch.to(device), out)

        loss.backward()
        optimizer.step()
        train_loss += loss
        
        c += 1
    scheduler.step(loss)
    print("Loss at epoch {} = {}".format(epoch, train_loss/c))
#     print("Last Loss = {}".format(loss.item()))

Loss at epoch 0 = 0.9014084935188293
Loss at epoch 1 = 0.7757490873336792
Loss at epoch 2 = 0.7329035997390747
Loss at epoch 3 = 0.7005599141120911
Loss at epoch 4 = 0.6877195835113525
Loss at epoch 5 = 0.6744323968887329
Loss at epoch 6 = 0.6600192189216614
Loss at epoch 7 = 0.6478566527366638
Loss at epoch 8 = 0.637294590473175
Loss at epoch 9 = 0.6241940855979919
Loss at epoch 10 = 0.6077831387519836
Loss at epoch 11 = 0.5996001362800598
Loss at epoch 12 = 0.5936069488525391
Loss at epoch 13 = 0.5903225541114807
Loss at epoch 14 = 0.5871611833572388
Loss at epoch 15 = 0.5847481489181519
Loss at epoch 16 = 0.5822124481201172
Loss at epoch 17 = 0.5801677703857422
Loss at epoch 18 = 0.5785619020462036
Loss at epoch 19 = 0.5775218605995178


In [16]:
ratings_test = test['rating'].values
userIds_test = test['userId'].values
itemIds_test = test['movieId'].values
pred = net.forward(torch.tensor(userIds_test).to(device),torch.tensor(itemIds_test).to(device)).cpu().detach().numpy()
pred = np.array([s[0] for s in pred])
MSE_test_nn = np.sum(np.square(ratings_test - pred))/ratings_test.shape[0]


3. Compare the Deep learning-based approach and basic Collaborative Filtering model.

In [17]:
print(f"basic Collaborative Filtering model MSE on test dataset = {MSE_test_MF}")
print(f"Deep learning model MSE on test dataset = {MSE_test_nn}")

basic Collaborative Filtering model MSE on test dataset = 0.7642871298737126
Deep learning model MSE on test dataset = 0.7134742950961608


Method that take a user id as an input and return a top 5 recommended movies for that given user.

In [18]:
def pred(user_id, method = 'MF'):
    if method == 'MF':
        p = P[user_id]
        scores = np.inner(p, Q)
    if method == 'NN':
        scores = net.forward(torch.tensor([user_id for _ in range(n_items)]).to(device),torch.tensor(list(range(n_items))).to(device)).cpu().detach().numpy()
        scores = np.array([s[0] for s in scores])
    ids = np.argsort(-np.array(scores))
    return ids[:5]